<a href="https://colab.research.google.com/github/pritampalit/ML4DQM/blob/master/Double_AE_Step_by_Step_PCA_AE_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [47]:
import pandas as pd
#from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
import json
#%matplotlib inline


#uploaded = files.upload()
#import io 
#df= pd.read_csv(io.BytesIO(uploaded['Tot_ZeroBias_UL2017_DataFrame_chargeInner_PXLayer_1.csv']))
#df= pd.read_csv('ZeroBias_2017UL_DataFrame_ChargeInnerLayer4.txt')
df= pd.read_csv('/content/drive/My Drive/Colab Notebooks/Tot_ZeroBias_UL2017_DataFrame_chargeInner_PXLayer_1.csv')
df['histo']=df['histo'].apply(literal_eval)

df.set_index(['fromrun','fromlumi'], inplace=True, drop=False)
df.sort_index(inplace=True)
print(df['histo'].shape)
print(df.tail())
print(df.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(225954,)
                  Unnamed: 0  ...                                              histo
fromrun fromlumi              ...                                                   
306462  95             67823  ...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
        96             67824  ...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
        97             67825  ...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
        98             67826  ...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
        99             67827  ...  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[5 rows x 10 columns]
(225954, 10)


In [48]:
import seaborn as sns
from pylab import rcParams
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from numpy.random import seed
seed(1)
import tensorflow
#from tensorflow import set_random_seed
tensorflow.random.set_seed(2)
#set_random_seed(2)
SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2
DATA_SPLIT_PCT_VALID = 0.1
rcParams['figure.figsize'] = 10, 8




In [49]:
#add Golden JSON labels to the DF
import json

def checkLS(run,ls):
    isok=False
    global jsondata

    if str(run) in jsondata.keys():
        for i in jsondata[str(run)]:
            if (ls>=i[0] and ls <=i[1]):
                isok=True
                return isok
        return   isok

#load the golden json file
jsondata={}
with open('/content/drive/My Drive/Colab Notebooks/GoldenJSON17.json') as json_file:
    jsondata = json.load(json_file)

df['labels']=False #initialize to false

for run in df['fromrun'].unique():
    for ls in df['fromlumi'][run]:
        df['labels'][run][ls]=checkLS(run,ls)

'''
def checkLSAnomaly(run,ls):
    isok_anomaly=True
    global jsondata
    
    if str(run) in jsondata.keys():
        for i in jsondata[str(run)]:
            if (ls>=i[0] and ls <=i[1]):
                isok_anomaly=False
                return isok_anomaly
        return isok_anomaly


df['anomaly']=True #initialize
for run in df['fromrun'].unique():
    for ls in df['fromlumi'][run]:  
        df['anomaly'][run][ls]=checkLSAnomaly(run,ls)
'''
#fname_bad = 'BAD_Tot_ZeroBias_UL2017_DataFrame_chargeInner_PXLayer_1.txt'
#fname_gold = 'GOLDEN_Tot_ZeroBias_UL2017_DataFrame_chargeInner_PXLayer_1.csv'
#df[df['labels']!=True].to_csv(fname_bad)
#df[df['labels']==True].to_csv(fname_gold)

#print(df[df['labels']!=True]) #to check against the Golden JSON
print(df[df['labels']==False].shape)
print(df[df['labels']==True].shape)
#print(df[df['anomaly']==True].shape)
#print(df[df['anomaly']==False].shape)
#print(df[df['anomaly']==False])
#print(df[df['anomaly']==True])
#print(df.shape)
#print(df['labels'])
#print(df['anomaly'])

df['type']=df['labels']
df['type']=df['type'].apply(lambda x:0 if x== True else 1)
df[df['type']==0]


(23655, 11)
(202299, 11)


Unnamed: 0  fromrun  ...  labels type
fromrun fromlumi                       ...             
297050  12              4535   297050  ...    True    0
        13              4536   297050  ...    True    0
        14             15690   297050  ...    True    0
        15             15691   297050  ...    True    0
        16              4537   297050  ...    True    0
...                      ...      ...  ...     ...  ...
306460  60             62547   306460  ...    True    0
        61             62548   306460  ...    True    0
        62             62549   306460  ...    True    0
        63             62550   306460  ...    True    0
        64             62551   306460  ...    True    0

[202299 rows x 12 columns]

In [50]:
df_train, df_test = train_test_split(df, test_size=DATA_SPLIT_PCT, random_state=SEED)
df_train, df_valid = train_test_split(df_train, test_size=DATA_SPLIT_PCT_VALID, random_state=SEED)

X_train_true = np.stack(df_train['histo'][df_train['labels']==True].values, axis=0)
X_train_false = np.stack(df_train['histo'][df_train['labels']==False].values, axis=0)

X_valid_true = np.stack(df_valid['histo'][df_valid['labels']==True].values, axis=0)
X_valid_false = np.stack(df_valid['histo'][df_valid['labels']==False].values, axis=0)


X_test_true = np.stack(df_test['histo'][df_test['labels']==True].values, axis=0)
X_test_false = np.stack(df_test['histo'][df_test['labels']==False].values, axis=0)

from sklearn.preprocessing import normalize

X_train_true_norm=normalize(X_train_true, norm='l1', axis=1) #normalise the sample, i.e the rows
X_train_false_norm=normalize(X_train_false, norm='l1', axis=1)

X_valid_true_norm=normalize(X_valid_true, norm='l1', axis=1) #normalise the sample, i.e the rows
X_valid_false_norm=normalize(X_valid_false, norm='l1', axis=1)

X_test_true_norm=normalize(X_test_true, norm='l1', axis=1) #normalise the sample, i.e the rows
X_test_false_norm=normalize(X_test_false, norm='l1', axis=1)




In [51]:
#build the model
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Dense, Layer, InputSpec
from keras.layers.advanced_activations import PReLU
from keras.models import Model, load_model
from keras import backend as K
from sklearn import decomposition
import scipy
import tensorflow as tf
from keras.constraints import UnitNorm, Constraint
from keras import regularizers, activations, initializers, constraints, Sequential

def mseTop10(y_true, y_pred):
    top_values, _ = tf.nn.top_k(K.square(y_pred - y_true), k=10, sorted=True)
    mean=K.mean(top_values, axis=-1)
    return mean

nb_epoch = 100
batch_size = 1000
input_dim = X_train_true_norm.shape[1] #num of predictor variables, 
encoding_dim = 3
#hidden_dim = int(encoding_dim / 2)
hidden_dim = 3
learning_rate = 1e-3

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", input_shape=(input_dim,), use_bias = True)  #, activity_regularizer=regularizers.l1(learning_rate))(input_layer)
#encoder = Dense(hidden_dim, activation="tanh")(encoder)
#decoder = Dense(hidden_dim, activation="tanh" )(encoder)
#decoder = DenseTied(encoding_dim, activation="tanh", tied_to=encoder, use_bias = True)(decoder)
decoder = Dense(input_dim, activation="tanh", use_bias = True)
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#autoencoder = Model(inputs=input_layer, outputs=decoder)
'''
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
'''

autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss=mseTop10,
                    optimizer='adam')
history = autoencoder.fit(X_train_true_norm, X_train_true_norm,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid_true_norm, X_valid_true_norm),
                    verbose=1).history


# Fit PCA
pca = decomposition.PCA(n_components=3)
pca.fit(X_train_true_norm)



Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 3)                 303       
_________________________________________________________________
dense_13 (Dense)             (None, 100)               400       
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________
Train on 145591 samples, validate on 16212 samples
Epoch 1/100
145591/145591 [==============================] - 1s 9us/step - loss: 3.0121e-04 - accuracy: 0.1699 - val_loss: 8.2446e-05 - val_accuracy: 0.1950
Epoch 2/100
145591/145591 [==============================] - 1s 8us/step - loss: 3.9325e-05 - accuracy: 0.2455 - val_loss: 2.0977e-05 - val_accuracy: 0.3282
Epoch 3/100
145591/145591 [==============================] - 1s 8us/step - loss: 1.7201e-05 - accuracy: 0.3432 - val_loss: 1.2850e-05 - val_accuracy: 0.3

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [52]:
#print(autoencoder.layers[0].get_weights()[0])
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 3.
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 2)  # W' in Figure 3.
print('Encoder weights \n', w_encoder)
print('Decoder weights \n', w_decoder)

Encoder weights 
 [[ 0.12  0.03 -0.07 -0.05 -0.05 -0.01 -0.01 -0.2  -0.11 -0.12 -0.13 -0.2
  -0.15 -0.15 -0.2  -0.16 -0.18 -0.15 -0.29 -0.08 -0.07 -0.08 -0.14 -0.05
   0.11  0.03  0.18  0.14  0.06  0.11  0.12  0.11  0.17  0.19  0.2   0.3
   0.21  0.22  0.13  0.03 -0.01 -0.07  0.1  -0.06  0.16  0.02  0.12  0.22
  -0.07  0.15  0.14 -0.09  0.02 -0.09  0.07  0.15  0.18  0.13 -0.03  0.11
  -0.09  0.18  0.04  0.01 -0.03  0.16  0.03  0.22  0.25  0.1  -0.09  0.11
   0.16  0.05  0.28  0.07  0.27  0.02  0.24 -0.08 -0.03 -0.13 -0.16 -0.13
   0.26 -0.04 -0.08  0.15 -0.17  0.14  0.19  0.01 -0.01  0.13  0.2  -0.21
  -0.04 -0.08  0.12  0.21]
 [-0.19 -0.33 -0.16 -0.04  0.01 -0.01  0.06  0.15  0.22  0.3   0.26  0.24
   0.11  0.19  0.19  0.02  0.03 -0.01 -0.14 -0.22 -0.17 -0.2  -0.31 -0.28
  -0.18 -0.24 -0.26 -0.13 -0.19 -0.09  0.01 -0.05 -0.06 -0.03  0.01  0.1
   0.01  0.12  0.24  0.14  0.24  0.09  0.08  0.05 -0.01  0.15  0.11  0.19
   0.03 -0.04  0.13  0.05  0.02  0.06  0.03  0.08 -0.02  0.15 -0.03  0

In [33]:
w_pca = pca.components_
np.round(np.dot(w_pca, w_pca.T), 3)

array([[ 1., -0., -0.],
       [-0.,  1., -0.],
       [-0., -0.,  1.]])

In [34]:
np.round(np.dot(w_encoder, w_encoder.T), 3)

array([[2.042, 0.159, 0.138],
       [0.159, 2.383, 0.25 ],
       [0.138, 0.25 , 1.539]], dtype=float32)

In [35]:
np.round(np.dot(w_decoder, w_decoder.T), 3)

array([[ 1.145, -0.099,  0.306],
       [-0.099,  0.868, -0.253],
       [ 0.306, -0.253,  2.699]], dtype=float32)

In [36]:
pca_features = pca.fit_transform(X_train_true_norm)
np.round(np.cov(pca_features.T), 5)

array([[ 2.43e-03, -0.00e+00, -0.00e+00],
       [-0.00e+00,  4.10e-04, -0.00e+00],
       [-0.00e+00, -0.00e+00,  4.00e-05]])

In [37]:
encoder_layer = Model(inputs=autoencoder.inputs, outputs=autoencoder.layers[0].output)
encoded_features = np.array(encoder_layer.predict(X_train_true_norm))
print('Encoded feature covariance\n', np.cov(encoded_features.T))

Encoded feature covariance
 [[ 0.00078289 -0.00044526 -0.00058556]
 [-0.00044526  0.00108872  0.00070461]
 [-0.00058556  0.00070461  0.00062688]]


In [38]:
print('PCA weights norm, \n', np.sum(w_pca ** 2, axis = 1))
print('Encoder weights norm, \n', np.sum(w_encoder ** 2, axis = 1))
print('Decoder weights norm, \n', np.sum(w_decoder ** 2, axis = 1))

PCA weights norm, 
 [1. 1. 1.]
Encoder weights norm, 
 [2.0417006 2.3830006 1.539    ]
Decoder weights norm, 
 [1.1450001 0.8684001 2.6994002]


In [39]:
train_predictions = autoencoder.predict(X_train_true_norm)
#print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
print('Train reconstrunction error\n', mseTop10(X_train_true_norm, train_predictions))
test_predictions = autoencoder.predict(X_test_true_norm)
#print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))
print('Test reconstrunction error\n', mseTop10(X_test_true_norm, test_predictions))

Train reconstrunction error
 tf.Tensor(
[1.67759913e-06 6.55572782e-07 1.32211385e-06 ... 9.11456652e-06
 8.98501991e-07 3.64721727e-06], shape=(145591,), dtype=float64)
Test reconstrunction error
 tf.Tensor(
[5.01978629e-06 8.02176342e-07 6.06355863e-07 ... 1.72796411e-06
 1.32675633e-06 2.74711861e-06], shape=(40496,), dtype=float64)


In [53]:
from keras import regularizers, activations, initializers, constraints, Sequential
class DenseTied(Layer):
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 tied_to=None,
                 **kwargs):
        self.tied_to = tied_to
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super().__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True
                
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        if self.tied_to is not None:
            self.kernel = K.transpose(self.tied_to.kernel)
            self._non_trainable_weights.append(self.kernel)
        else:
            self.kernel = self.add_weight(shape=(input_dim, self.units),
                                          initializer=self.kernel_initializer,
                                          name='kernel',
                                          regularizer=self.kernel_regularizer,
                                          constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def call(self, inputs):
        output = K.dot(inputs, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

In [54]:
encoder = Dense(encoding_dim, activation="tanh", input_shape=(input_dim,), use_bias = True)  #, activity_regularizer=regularizers.l1(learning_rate))(input_layer)
#encoder = Dense(hidden_dim, activation="tanh")(encoder)
#decoder = Dense(hidden_dim, activation="tanh" )(encoder)
#decoder = DenseTied(encoding_dim, activation="tanh", tied_to=encoder, use_bias = True)(decoder)
decoder = DenseTied(input_dim, activation="tanh", tied_to=encoder, use_bias = True)
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#autoencoder = Model(inputs=input_layer, outputs=decoder)
'''
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
'''

autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss=mseTop10,
                    optimizer='adam')
history = autoencoder.fit(X_train_true_norm, X_train_true_norm,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid_true_norm, X_valid_true_norm),
                    verbose=1).history



Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 3)                 303       
_________________________________________________________________
dense_tied_4 (DenseTied)     (None, 100)               703       
Total params: 703
Trainable params: 403
Non-trainable params: 300
_________________________________________________________________
Train on 145591 samples, validate on 16212 samples
Epoch 1/100
145591/145591 [==============================] - 1s 9us/step - loss: 2.6605e-04 - accuracy: 0.0609 - val_loss: 8.6784e-05 - val_accuracy: 0.1553
Epoch 2/100
145591/145591 [==============================] - 1s 8us/step - loss: 4.8829e-05 - accuracy: 0.2194 - val_loss: 2.6493e-05 - val_accuracy: 0.2598
Epoch 3/100
145591/145591 [==============================] - 1s 8us/step - loss: 2.1488e-05 - accuracy: 0.2716 - val_loss: 1.7073e-05 - val_accuracy: 0

In [57]:
#w_encoder = np.round(np.transpose(autoencoder.layers[0].get_weights()[0]), 3)
#w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 3)
#print('Encoder weights\n', w_encoder)
#print('Decoder weights\n', w_decoder)
#print(autoencoder.layers[0].get_weights()[0])
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 3.
#w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 2)  # W' in Figure 3.
print('Encoder weights \n', w_encoder)
#print('Decoder weights \n', w_decoder)
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 3)

Encoder weights 
 [[ 0.04  0.1   0.13  0.13  0.11  0.07  0.01 -0.06 -0.12 -0.19 -0.24 -0.29
  -0.32 -0.32 -0.29 -0.24 -0.18 -0.12 -0.05  0.02  0.08  0.13  0.16  0.19
   0.2   0.21  0.21  0.2   0.18  0.16  0.14  0.11  0.08  0.06  0.03  0.01
   0.   -0.   -0.01 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.01 -0.01 -0.01
  -0.    0.    0.01  0.01  0.01  0.01  0.   -0.    0.01  0.01  0.01  0.01
   0.01  0.01 -0.01 -0.01  0.01 -0.01  0.    0.01  0.01 -0.02  0.01 -0.02
  -0.01  0.01  0.01 -0.01  0.01  0.    0.   -0.02 -0.01 -0.02  0.    0.01
  -0.02  0.01  0.01  0.01  0.01 -0.02 -0.01 -0.02 -0.01 -0.    0.01 -0.02
  -0.01  0.01 -0.01  0.01]
 [ 0.04  0.14  0.21  0.25  0.28  0.29  0.27  0.25  0.21  0.16  0.1   0.04
  -0.04 -0.1  -0.15 -0.19 -0.22 -0.23 -0.23 -0.23 -0.21 -0.2  -0.19 -0.17
  -0.15 -0.12 -0.1  -0.08 -0.06 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02
   0.03  0.04  0.04  0.04  0.04  0.04  0.03  0.03  0.03  0.03  0.03  0.03
   0.03  0.03  0.03  0.02  0.02  0.02  0.02  0.01  0.01  0.01  0.01

AttributeError: ignored

In [59]:
b_encoder = np.round(np.transpose(autoencoder.layers[0].get_weights()[1]), 3)
#b_decoder = np.round(np.transpose(autoencoder.layers[1].get_weights()[0]), 3)
print('Encoder bias\n', b_encoder)
#print('Decoder bias\n', b_decoder)

Encoder bias
 [ 0.016  0.051 -0.045]


In [60]:
class WeightsOrthogonalityConstraint (Constraint):
    def __init__(self, encoding_dim, weightage = 1.0, axis = 0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage
        self.axis = axis
        
    def weights_orthogonality(self, w):
        if(self.axis==1):
            w = K.transpose(w)
        if(self.encoding_dim > 1):
            m = K.dot(K.transpose(w), w) - K.eye(self.encoding_dim)
            return self.weightage * K.sqrt(K.sum(K.square(m)))
        else:
            m = K.sum(w ** 2) - 1.
            return m

    def __call__(self, w):
        return self.weights_orthogonality(w)

In [65]:
encoder = Dense(encoding_dim, activation="tanh", input_shape=(input_dim,), use_bias = True,   kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0) )
#encoder = Dense(hidden_dim, activation="tanh")(encoder)
#decoder = Dense(hidden_dim, activation="tanh" )(encoder)
#decoder = DenseTied(encoding_dim, activation="tanh", tied_to=encoder, use_bias = True)(decoder)
decoder = Dense(input_dim, activation="tanh", use_bias = True,
kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=1))

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#autoencoder = Model(inputs=input_layer, outputs=decoder)
'''
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
'''

autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss=mseTop10,
                    optimizer='adam')
history = autoencoder.fit(X_train_true_norm, X_train_true_norm,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid_true_norm, X_valid_true_norm),
                    verbose=1).history



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 3)                 303       
_________________________________________________________________
dense_17 (Dense)             (None, 100)               400       
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________
Train on 145591 samples, validate on 16212 samples
Epoch 1/100
145591/145591 [==============================] - 1s 10us/step - loss: 0.4997 - accuracy: 0.1056 - val_loss: 0.0045 - val_accuracy: 0.1424
Epoch 2/100
145591/145591 [==============================] - 1s 9us/step - loss: 0.0036 - accuracy: 0.1447 - val_loss: 0.0038 - val_accuracy: 0.1490
Epoch 3/100
145591/145591 [==============================] - 1s 9us/step - loss: 0.0046 - accuracy: 0.1450 - val_loss: 0.0041 - val_accuracy: 0.1405
Epoch 4/100
145591/

In [66]:
w_encoder = autoencoder.layers[0].get_weights()[0]
print('Encoder weights dot product\n', np.round(np.dot(w_encoder.T, w_encoder), 2))

w_decoder = autoencoder.layers[1].get_weights()[0]
print('Decoder weights dot product\n', np.round(np.dot(w_decoder, w_decoder.T), 2))

Encoder weights dot product
 [[ 1. -0. -0.]
 [-0.  1. -0.]
 [-0. -0.  1.]]
Decoder weights dot product
 [[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0. -0.  1.]]


In [67]:
class UncorrelatedFeaturesConstraint (Constraint):
    
    def __init__(self, encoding_dim, weightage = 1.0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage
    
    def get_covariance(self, x):
        x_centered_list = []

        for i in range(self.encoding_dim):
            x_centered_list.append(x[:, i] - K.mean(x[:, i]))
        
        x_centered = tf.stack(x_centered_list)
        covariance = K.dot(x_centered, K.transpose(x_centered)) / tf.cast(x_centered.get_shape()[0], tf.float32)
        
        return covariance
            
    # Constraint penalty
    def uncorrelated_feature(self, x):
        if(self.encoding_dim <= 1):
            return 0.0
        else:
            output = K.sum(K.square(
                self.covariance - tf.math.multiply(self.covariance, K.eye(self.encoding_dim))))
            return output

    def __call__(self, x):
        self.covariance = self.get_covariance(x)
        return self.weightage * self.uncorrelated_feature(x)

In [68]:
encoder = Dense(encoding_dim, activation="tanh", input_shape=(input_dim,), use_bias = True,   
                activity_regularizer=UncorrelatedFeaturesConstraint(encoding_dim, weightage = 1.) )
#encoder = Dense(hidden_dim, activation="tanh")(encoder)
#decoder = Dense(hidden_dim, activation="tanh" )(encoder)
#decoder = DenseTied(encoding_dim, activation="tanh", tied_to=encoder, use_bias = True)(decoder)
decoder = Dense(input_dim, activation="tanh", use_bias = True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#autoencoder = Model(inputs=input_layer, outputs=decoder)
'''
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
'''

autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss=mseTop10,
                    optimizer='adam')
history = autoencoder.fit(X_train_true_norm, X_train_true_norm,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid_true_norm, X_valid_true_norm),
                    verbose=1).history



Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 3)                 303       
_________________________________________________________________
dense_19 (Dense)             (None, 100)               400       
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________
Train on 145591 samples, validate on 16212 samples
Epoch 1/100
145591/145591 [==============================] - 1s 10us/step - loss: 3.3556e-04 - accuracy: 0.1279 - val_loss: 1.4960e-04 - val_accuracy: 0.1887
Epoch 2/100
145591/145591 [==============================] - 1s 9us/step - loss: 1.3698e-04 - accuracy: 0.1771 - val_loss: 1.2092e-04 - val_accuracy: 0.1787
Epoch 3/100
145591/145591 [==============================] - 1s 9us/step - loss: 9.9506e-05 - accuracy: 0.1682 - val_loss: 7.7610e-05 - val_accuracy: 0.

In [70]:
encoder_layer = Model(inputs=autoencoder.inputs, outputs=autoencoder.layers[0].output)
encoded_features = np.array(encoder_layer.predict(X_train_true_norm))
print('Encoded feature covariance\n', np.round(np.cov(encoded_features.T), 3))

Encoded feature covariance
 [[ 0.  0. -0.]
 [ 0.  0.  0.]
 [-0.  0.  0.]]


In [71]:
encoder = Dense(encoding_dim, activation="tanh", input_shape=(input_dim,), use_bias = True,   
                kernel_constraint=UnitNorm(axis=0) )
#encoder = Dense(hidden_dim, activation="tanh")(encoder)
#decoder = Dense(hidden_dim, activation="tanh" )(encoder)
#decoder = DenseTied(encoding_dim, activation="tanh", tied_to=encoder, use_bias = True)(decoder)
decoder = Dense(input_dim, activation="tanh", use_bias = True,
                kernel_constraint=UnitNorm(axis=1))

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#autoencoder = Model(inputs=input_layer, outputs=decoder)
'''
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
'''

autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss=mseTop10,
                    optimizer='adam')
history = autoencoder.fit(X_train_true_norm, X_train_true_norm,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid_true_norm, X_valid_true_norm),
                    verbose=1).history



Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 3)                 303       
_________________________________________________________________
dense_21 (Dense)             (None, 100)               400       
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________
Train on 145591 samples, validate on 16212 samples
Epoch 1/100
145591/145591 [==============================] - 1s 9us/step - loss: 3.2112e-04 - accuracy: 0.0377 - val_loss: 1.0544e-04 - val_accuracy: 0.0019
Epoch 2/100
145591/145591 [==============================] - 1s 8us/step - loss: 5.8329e-05 - accuracy: 0.0812 - val_loss: 3.1731e-05 - val_accuracy: 0.1839
Epoch 3/100
145591/145591 [==============================] - 1s 8us/step - loss: 2.3038e-05 - accuracy: 0.2492 - val_loss: 1.5791e-05 - val_accuracy: 0.

In [74]:
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 3.
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 2)  # W' in Figure 3.
print('Encoder weights norm, \n', np.round(np.sum(w_encoder ** 2, axis = 1),3))
print('Decoder weights norm, \n', np.round(np.sum(w_decoder ** 2, axis = 1),3))

Encoder weights norm, 
 [1.    1.002 0.999]
Decoder weights norm, 
 [1.006 0.998 0.995]
